In [1]:
import numpy as np

In [64]:
ind_to_ph = []
with open('phonemes.txt','r') as file:
    for line in file:
        ind_to_ph.append(line.split()[0])
ind_to_ph.append(' ')
print(ind_to_ph)

['AA', 'AE', 'AH', 'AO', 'AW', 'AY', 'B', 'CH', 'D', 'DH', 'EH', 'ER', 'EY', 'F', 'G', 'HH', 'IH', 'IY', 'JH', 'K', 'L', 'M', 'N', 'NG', 'OW', 'OY', 'P', 'R', 'S', 'SH', 'T', 'TH', 'UH', 'UW', 'V', 'W', 'Y', 'Z', 'ZH', ' ']


In [67]:
ind_to_char = "ABCDEFGHIJKLMNOPQRSTUVWXYZ "

In [68]:
num_lines = 0
maxlen = 0
maxph = 0
longword = ''
with open('train.txt', 'r') as datafile:
    for line in datafile:
        maxlen = max(maxlen, len(line.split()[0]))
        if maxlen == len(line.split()[0]):
            longword = line.split()[0]
        phonemes = line.split()[1]
        phonemes = phonemes.split('_')
        maxph = max(maxph, len(phonemes))
        num_lines += 1
print(num_lines, maxlen, maxph, longword)

83194 34 32 SUPERCALIFRAGILISTICEXPIALIDOCIOUS


In [69]:
x = np.zeros((num_lines, maxlen, len(ind_to_char)), dtype=np.bool)
y = np.zeros((num_lines, maxph, len(ind_to_ph)), dtype=np.bool)
with open('train.txt', 'r') as datafile:
    lines = 0
    for line in datafile:
        word = line.split()[0]
        index = 0
        phonemes = line.split()[1]
        phonemes = phonemes.split('_')
        for phoneme in phonemes:
            y[lines, index, ind_to_ph.index(phoneme)] = 1
            index += 1
        for i in range(index, maxph):
            y[lines, i, ind_to_ph.index(' ')] = 1
        index = 0
        for sym in word:
            if sym in ind_to_char:
                x[lines, index, ind_to_char.index(sym)] = 1
            index += 1
        for i in range(index, maxlen):
            x[lines, i, ind_to_char.index(' ')] = 1
        lines += 1

In [74]:
from __future__ import print_function
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.models import load_model
from keras import layers

In [75]:
to_val = 10000
x_train, x_test = x[:-to_val], x[-to_val:]
y_train, y_test = y[:-to_val], y[-to_val:]

In [76]:
print(x_test.shape)

(10000, 34, 27)


In [77]:
RNN = layers.LSTM
HIDDEN_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(maxlen, len(ind_to_char))))
model.add(layers.RepeatVector(maxph))
for _ in range(LAYERS):
    model.add(Bidirectional(RNN(HIDDEN_SIZE, return_sequences=True)))

model.add(layers.TimeDistributed(layers.Dense(len(ind_to_ph))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 128)               79872     
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 32, 128)           0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 32, 256)           263168    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 32, 40)            10280     
_________________________________________________________________
activation_5 (Activation)    (None, 32, 40)            0         
Total params: 353,320.0
Trainable params: 353,320.0
Non-trainable params: 0.0
_________________________________________________________________


In [105]:
def decode(table, x, calc_argmax=True):
    if calc_argmax:
        x = x.argmax(axis=-1)
    return '_'.join(table[x] for x in x)

In [85]:
for i in range(1):
    BATCH_SIZE = 128
    print('Train...')
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              verbose=2,
              validation_data=(x_test, y_test))

    for i in range(10):
            ind = np.random.randint(0, len(x_test))
            rowx, rowy = x_test[np.array([ind])], y_test[np.array([ind])]
            preds = model.predict_classes(rowx, verbose=0)
            q = decode(ind_to_char,rowx[0])
            correct = decode(ind_to_ph,rowy[0])
            guess = decode(ind_to_ph,preds[0], calc_argmax=False)
            print('Q', q)
            print('T', correct)
            if correct == guess:
                print('OK')
            else:
                print('FAIL', end=" ")
            print(guess)
            print('-'*100)

Train...
Train on 73194 samples, validate on 10000 samples
Epoch 1/1
474s - loss: 0.0897 - acc: 0.9711 - val_loss: 0.1370 - val_acc: 0.9584
Q D_R_I_P_A_D_R_Y_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
T D_R_IH_P_D_R_AY_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
FAIL S_R_IH_P_P_R_IY_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
----------------------------------------------------------------------------------------------------
Q C_O_N_T_A_C_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
T K_AA_N_T_AE_K_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
OK
K_AA_N_T_AE_K_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
----------------------------------------------------------------------------------------------------
Q D_R_I_P_A_D_R_Y_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
T D_R_IH_P_D_R_AY_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
FAIL S_R_IH_P_P_R_IY_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
---------------------------------

In [86]:
model.save('test69')

In [12]:
model2 = load_model('test1')

In [2]:
import pandas as pd

In [89]:
to_pred = pd.read_csv('test.csv')

In [90]:
# 1..41597
val_test = np.zeros((41597, 34, 27), dtype=bool)
lines = 0
for word in to_pred['Word']:
    index = 0
    for sym in word:
        if sym in ind_to_char:
            val_test[lines, index, ind_to_char.index(sym)] = 1
        index += 1
    for i in range(index, maxlen):
        val_test[lines, i, ind_to_char.index(' ')] = 1
    lines += 1
    if lines == 41597:
        break

In [93]:
two_els = val_test[:3]

In [ ]:
test_answers = model.predict_classes(val_test)

35488/41597 [========================>.....] - ETA: 28s    64/41597 [..............................] - ETA: 185s

In [123]:
with open('test.txt', 'w') as f:
    f.write('Id,Transcription\n')
    for ind in range(len(test_answers)):
        f.write(str(ind + 1) +',' 
                + '_'.join(ind_to_ph[x] for x in test_answers[ind, :test_answers[ind].tolist().index(39)]) + '\n')
        